In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, recall_score
from sklearn.feature_selection import RFECV, SelectKBest
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC

from hyperopt import fmin, tpe, STATUS_OK, Trials, SparkTrials, hp, space_eval
from imblearn.over_sampling import RandomOverSampler

from joblib import dump, load
import pyspark
import os
import time

In [2]:
complete_data = pd.read_csv('Data/complete_data.csv')
descriptors = pd.read_csv('Data/descriptors.csv')
class_data = pd.read_csv('Data/class_data.csv')

In [3]:
x = np.array(descriptors)
X = np.nan_to_num(x.astype(np.float32))

y = np.array(class_data['2-Class'])
y = (y + 1) / 2

/var/folders/wl/vkrd4x2j7fs5_ybd9y74b2xm0000gn/T/ipykernel_60805/4091702182.py:2: RuntimeWarning: overflow encountered in cast
  X = np.nan_to_num(x.astype(np.float32))


In [4]:
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X, y)

train_acc_matrix = np.zeros(20)
test_acc_matrix = np.zeros(20)
delta_matrix = np.zeros(20)
train_rec_matrix = np.zeros(20)
test_rec_matrix = np.zeros(20)

features = np.array([10 * (i + 1) for i in range(20)])
features[19] = features[19] - 1

In [6]:
for j in range(20):
    k = (j + 1) * 10 if j != 19 else 199

    X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42,
                                                        stratify=y_over)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    selector = SelectKBest(k=k)
    X_train = selector.fit_transform(X_train, y_train)
    X_test = selector.transform(X_test)

    best_j = load(f'SVM_imbalance_recall_loop_3/best_model_recall_loop_{j}.joblib')
    kernel = ['linear', 'poly', 'rbf']
    degree = [2, 3, 4, 5]

    model = SVC(random_state=42, C=best_j['C'], kernel=kernel[best_j['kernel']], gamma=best_j['gamma'],
                degree=degree[best_j['degree']], shrinking=best_j['shrinking'])

    # model = load(f'SVM_imbalance_recall_loop/test_model_recall_loop_{i}.joblib')
    model.fit(X_train, y_train)

    train_acc = accuracy_score(y_train, model.predict(X_train))
    test_acc = accuracy_score(y_test, model.predict(X_test))

    train_rec = recall_score(y_train, model.predict(X_train))
    test_rec = recall_score(y_test, model.predict(X_test))

    train_acc_matrix[j] = train_acc
    test_acc_matrix[j] = test_acc
    delta_matrix[j] = abs(train_acc - test_acc)
    train_rec_matrix[j] = train_rec
    test_rec_matrix[j] = test_rec

/Users/aronalagaraj/miniconda3/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 57  70 126 144 146 148 150 151 162 166 169 173 175 185 187 191 192 196] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/aronalagaraj/miniconda3/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/Users/aronalagaraj/miniconda3/lib/python3.11/site-packages/sklearn/utils/_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/Users/aronalagaraj/miniconda3/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 57  70 126 144 146 148 150 151 162 166 169 173 175 185 187 191 192 196] are constant.
  warnings.warn("Features %s are co

In [7]:
print(train_acc_matrix, '\n')
print(test_acc_matrix, '\n')
print(delta_matrix, '\n')
print(train_rec_matrix, '\n')
print(test_rec_matrix, '\n')

[0.97818792 0.99161074 0.99496644 0.98993289 0.99496644 0.99496644
 0.99496644 0.99496644 0.98154362 0.99496644 0.99496644 0.99496644
 0.99496644 0.99496644 0.99496644 0.99496644 0.99496644 0.99496644
 0.99496644 0.99496644] 

[0.88       0.87333333 0.86666667 0.87333333 0.88       0.88
 0.88       0.88       0.88666667 0.88       0.88       0.88
 0.88       0.88       0.88       0.87333333 0.88       0.88
 0.88       0.87333333] 

[0.09818792 0.1182774  0.12829978 0.11659955 0.11496644 0.11496644
 0.11496644 0.11496644 0.09487696 0.11496644 0.11496644 0.11496644
 0.11496644 0.11496644 0.11496644 0.12163311 0.11496644 0.11496644
 0.11496644 0.12163311] 

[0.97315436 0.99328859 1.         0.98993289 1.         1.
 1.         1.         0.97651007 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.        ] 

[0.94666667 0.97333333 0.96       0.98666667 1.         1.
 1.         1.         0.96       1.         1.         1.
 1.         1.  

In [ ]:
i = 5
k = (i + 1) * 10 if i != 19 else 199

X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, random_state=42, stratify=y_over)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

selector = SelectKBest(k=k)
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)

best_j = load(f'SVM_imbalance_recall_loop_delta_1/best_model_recall_loop_{i}.joblib')
kernel = ['linear', 'poly', 'rbf']
degree = [2, 3, 4, 5]

model = SVC(random_state=42, C=best_j['C'], kernel=kernel[best_j['kernel']], gamma=best_j['gamma'],
            degree=degree[best_j['degree']], shrinking=bool(best_j['shrinking']))

# model = load(f'SVM_imbalance_recall_loop/test_model_recall_loop_{j}.joblib')
model.fit(X_train, y_train)

train_acc = accuracy_score(y_train, model.predict(X_train))
test_acc = accuracy_score(y_test, model.predict(X_test))
train_rec = recall_score(y_train, model.predict(X_train))
test_rec = recall_score(y_test, model.predict(X_test))

print(f'Train Accuracy: {train_acc}')
print(f'Test Accuracy: {test_acc}')

print(f'Train Recall: {train_rec}')
print(f'Test Recall: {test_rec}')

In [ ]:
print(descriptors.columns[selector.get_support()])